# NYC Parking Ticket Analysis - Case Study

New York City is a thriving metropolis. Just like most other metros its size, one of the biggest problems its citizens face is parking. The classic combination of a huge number of cars and cramped geography leads to a huge number of parking tickets.

In an attempt to scientifically analyse this phenomenon, the NYC Police Department has collected data for parking tickets. Of these, the data files for multiple years are publicly available on Kaggle. We will try and perform some exploratory analysis on a part of this data. Spark will allow us to analyse the full files at high speeds as opposed to taking a series of random samples that will approximate the population. For the scope of this analysis, we will analyse the parking tickets over the year 2017. 

In [1]:
# Initiation of Spark Session 
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("NYC_Parking") \
    .getOrCreate()

In [2]:
# The data for this case study has been placed in HDFS at the following path:
# '/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv'
# Read ticketing data
parking = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

In [3]:
# Check if data is read right 
parking.show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
|    8478629828| 66623ME|         

In [4]:
# Check the schema
parking.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Violation Time: string (nullable = true)



In [20]:
# Filter only data for 2017
from pyspark.sql.functions import *
parking_2017=parking.where(year('Issue Date')==2017)

In [21]:
# Check data now
parking_2017.show(5)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|         1120A|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|         0852P|
|    1407740258| 2513JMG|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|               106|            106|         0015A|
|    1413656420|T672371C|         

### Examine Data  

In [22]:
# Q1. Find the total number of tickets for the year.

parking_2017.count()

# 5,431,918 tickets issued in 2017.

5431918

In [23]:
# Q2. Find out the number of unique states from where the cars that got parking tickets came. 
#     (Hint: Use the column 'Registration State'.)
#     There is a numeric entry '99' in the column, which should be corrected. 
#     Replace it with the state having the maximum entries. Provide the number of unique states again.

parking_2017.select('Registration State').distinct().count()

# 65 unique states.

65

In [24]:
# View the unique states with ticket count
parking_2017.groupby('Registration State').count().sort(desc("count")).show(65)

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|4273951|
|                NJ| 475825|
|                PA| 140286|
|                CT|  70403|
|                FL|  69468|
|                IN|  45525|
|                MA|  38941|
|                VA|  34367|
|                MD|  30213|
|                NC|  27152|
|                TX|  18827|
|                IL|  18666|
|                GA|  17537|
|                99|  16055|
|                AZ|  12379|
|                OH|  12281|
|                CA|  12153|
|                ME|  10806|
|                SC|  10395|
|                MN|  10083|
|                OK|   9088|
|                TN|   8514|
|                DE|   7905|
|                MI|   7231|
|                RI|   5814|
|                NH|   4119|
|                VT|   3683|
|                AL|   3178|
|                WA|   3052|
|                OR|   2622|
|                MO|   2483|
|             

In [25]:
# Replacing 99 with NY. NY is the state with most tickets.
parking_2017=parking_2017.withColumn('Registration State', regexp_replace('Registration State', '99', 'NY'))

In [26]:
# Get the unique state count after correction
parking_2017.select('Registration State').distinct().count()

64

In [28]:
# Get all the registration state details
parking_2017.groupby('Registration State').count().sort(desc("count")).show()

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|4290006|
|                NJ| 475825|
|                PA| 140286|
|                CT|  70403|
|                FL|  69468|
|                IN|  45525|
|                MA|  38941|
|                VA|  34367|
|                MD|  30213|
|                NC|  27152|
|                TX|  18827|
|                IL|  18666|
|                GA|  17537|
|                AZ|  12379|
|                OH|  12281|
|                CA|  12153|
|                ME|  10806|
|                SC|  10395|
|                MN|  10083|
|                OK|   9088|
+------------------+-------+
only showing top 20 rows



### Aggregation Tasks

In [29]:
# Q1. How often does each violation code occur? Display the frequency of the top five violation codes.
parking_2017.groupby('Violation Code').count().collect()

[Row(Violation Code=31, count=80593),
 Row(Violation Code=85, count=9316),
 Row(Violation Code=65, count=26),
 Row(Violation Code=53, count=19488),
 Row(Violation Code=78, count=26776),
 Row(Violation Code=34, count=11),
 Row(Violation Code=81, count=14),
 Row(Violation Code=28, count=5),
 Row(Violation Code=76, count=18),
 Row(Violation Code=27, count=3039),
 Row(Violation Code=26, count=660),
 Row(Violation Code=44, count=4),
 Row(Violation Code=12, count=53),
 Row(Violation Code=91, count=433),
 Row(Violation Code=22, count=81),
 Row(Violation Code=93, count=8),
 Row(Violation Code=47, count=65440),
 Row(Violation Code=1, count=674),
 Row(Violation Code=52, count=1001),
 Row(Violation Code=13, count=11673),
 Row(Violation Code=16, count=74790),
 Row(Violation Code=6, count=192),
 Row(Violation Code=86, count=6),
 Row(Violation Code=3, count=407),
 Row(Violation Code=20, count=319646),
 Row(Violation Code=40, count=277184),
 Row(Violation Code=94, count=199),
 Row(Violation Code=57, 

In [31]:
# Displaying top 5 violation codes based on ticket count. 
parking_2017.groupby('Violation Code').count().sort(desc("count")).show(5)

# Violation code 21 is top in the list. 

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+
only showing top 5 rows



In [32]:
# Q2. How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? 
#     (Hint: Find the top 5 for both.)

##Vehicle Body Type
parking_2017.groupby('Vehicle Body Type').count().sort(desc("count")).show(5)

## SUBN is the top.

+-----------------+-------+
|Vehicle Body Type|  count|
+-----------------+-------+
|             SUBN|1883954|
|             4DSD|1547312|
|              VAN| 724029|
|             DELV| 358984|
|              SDN| 194197|
+-----------------+-------+
only showing top 5 rows



In [33]:
# Vehicle Make
parking_2017.groupby('Vehicle Make').count().sort(desc("count")).show(5)

##FORD is the top.

+------------+------+
|Vehicle Make| count|
+------------+------+
|        FORD|636844|
|       TOYOT|605291|
|       HONDA|538884|
|       NISSA|462017|
|       CHEVR|356032|
+------------+------+
only showing top 5 rows



In [35]:
# Q3. A precinct is a police station that has a certain zone of the city under its command. 
#     Find the (5 highest) frequencies of tickets for each of the following:
#     1. Violation Precinct' (This is the precinct of the zone where the violation occurred). 
#     Using this, can you draw any insights for parking violations in any specific areas of the city?
#     2.'Issuer Precinct' (This is the precinct that issued the ticket.)
#     Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'. 
#     These are erroneous entries. Hence, you need to provide the records for five correct precincts. (Hint: Print the top six entries after sorting.)

parking_2017.groupby('Violation Precinct').count().sort(desc("count")).show(6)

# 0 is incorrect data
# Violation Precinct 19 is the top precinct where more violations occurred

+------------------+------+
|Violation Precinct| count|
+------------------+------+
|                 0|925596|
|                19|274445|
|                14|203553|
|                 1|174702|
|                18|169131|
|               114|147444|
+------------------+------+
only showing top 6 rows



In [36]:
# Find the frequencies of ticket based on 'Issuer Precinct' - top 5 
parking_2017.groupby('Issuer Precinct').count().sort(desc("count")).show(6)

# 19 is the top Issue Precinct where violation occurred. 

+---------------+-------+
|Issuer Precinct|  count|
+---------------+-------+
|              0|1078406|
|             19| 266961|
|             14| 200495|
|              1| 168740|
|             18| 162994|
|            114| 144054|
+---------------+-------+
only showing top 6 rows



In [37]:
# Q4. Find the violation code frequencies for three precincts that have issued the most number of tickets. 
#     Do these precinct zones have an exceptionally high frequency of certain violation codes? 
#     Are these codes common across precincts? 
#    (Hint: In the SQL view, use the 'where' attribute to filter among three precincts.)

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

# Create a SQL view.
parking_2017.createOrReplaceTempView("parkingTable")

# after registering temp table you can run your sql queries
df=spark.sql('SELECT `Issuer Precinct`,`Violation Code`, count(`Violation Code`) as countval FROM parkingTable where `Issuer Precinct` in (19,14,1) group by `Issuer Precinct`,`Violation Code` order by `Issuer Precinct`,count(`Violation Code`) desc')

In [38]:
df.show(3)

+---------------+--------------+--------+
|Issuer Precinct|Violation Code|countval|
+---------------+--------------+--------+
|              1|            14|   38354|
|              1|            16|   19081|
|              1|            20|   15408|
+---------------+--------------+--------+
only showing top 3 rows



In [39]:
# Use window function to get the top counts per issuer precinct

window = Window.partitionBy(df['Issuer Precinct']).orderBy(df['countval'].desc())
df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 5).show()

# 46, 38 , 14 are seen as common violation code 

+---------------+--------------+--------+----+
|Issuer Precinct|Violation Code|countval|rank|
+---------------+--------------+--------+----+
|              1|            14|   38354|   1|
|              1|            16|   19081|   2|
|              1|            20|   15408|   3|
|              1|            46|   12745|   4|
|              1|            38|    8535|   5|
|             19|            46|   48445|   1|
|             19|            38|   36386|   2|
|             19|            37|   36056|   3|
|             19|            14|   29797|   4|
|             19|            21|   28415|   5|
|             14|            14|   45036|   1|
|             14|            69|   30464|   2|
|             14|            31|   22555|   3|
|             14|            47|   18364|   4|
|             14|            42|   10027|   5|
+---------------+--------------+--------+----+



In [40]:
# Q5. Find out the properties of parking violations across different times of the day

from pyspark.sql.functions import unix_timestamp, to_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import *


# Drop records with null value
parking_nonull = parking_2017.na.drop()

# convert Issue Date to date format. 
parking_nonull=parking_nonull.withColumn("Date", (col("Issue Date").cast("date")))

# concat m to the violation time.
# convert Violation time to a valid timestamp 
parking_nonull = parking_nonull.withColumn('Violation Time', regexp_replace(col('Violation Time'), '^00', '12'))
parking_nonull=parking_nonull.withColumn("Violation Time",concat(col("Violation Time"),lit("M")))
parking_nonull=parking_nonull.withColumn("ViolationTime_Rd", from_unixtime(unix_timestamp(col('Violation Time'), 'hhmmaa'), 'HH:mm:ss'))

In [41]:
parking_nonull.show(3)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+----------+----------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|      Date|ViolationTime_Rd|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+----------+----------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|        1120AM|2017-06-14|        11:20:00|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|        0852PM|2017-06-13|        20:52:00|
|    1407740258| 2513JMG|                NY|2017-01-11 

In [42]:
# Segregation of 24 hours into 6 buckets and categorise the violations into 1 bucket.
# B1 : 0 -4 
# B2 : 5- 8   
# B3 : 9 - 12  
# B4 : 13 - 16  
# B5 : 17 - 20  
# B6 : 21 - 24

parking_nonull=parking_nonull.selectExpr("*","CASE WHEN hour(ViolationTime_Rd) >= 21 THEN  'B6' WHEN hour(ViolationTime_Rd) >= 17 and hour(ViolationTime_Rd) <= 20  THEN  'B5' WHEN hour(ViolationTime_Rd) >= 13 and hour(ViolationTime_Rd) <= 16  THEN  'B4' WHEN hour(ViolationTime_Rd) >= 9 and hour(ViolationTime_Rd) <= 12  THEN  'B3' WHEN hour(ViolationTime_Rd) >= 5 and hour(ViolationTime_Rd) <= 8  THEN  'B2' ELSE 'B1' END AS Hour_Frequency")

In [43]:
parking_nonull.show(3)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+----------+----------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|      Date|ViolationTime_Rd|Hour_Frequency|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+----------+----------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|        1120AM|2017-06-14|        11:20:00|            B3|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|        0852PM|2017-06-13|        20:52

In [44]:
# Finding the top 24 hour bucket with the ticket count
parking_nonull.groupby('Hour_Frequency').count().sort(desc("count")).show()

# most of the faults occur during 9 AM to 4 PM .

+--------------+-------+
|Hour_Frequency|  count|
+--------------+-------+
|            B3|2169845|
|            B4|1625806|
|            B2| 939173|
|            B5| 390776|
|            B1| 179184|
|            B6| 127134|
+--------------+-------+



In [45]:
#  For each of 24 hr bucket/ groups, find the three most commonly occurring violations.
parking_nonull.createOrReplaceTempView("parkingTable2")
df2=spark.sql('SELECT `Hour_Frequency`,`Violation Code`, count(`Violation Code`) as countval FROM parkingTable2 group by `Hour_Frequency`,`Violation Code` order by count(`Violation Code`) desc')
window2 = Window.partitionBy(df2['Hour_Frequency']).orderBy(df2['countval'].desc())
df2.select('*', rank().over(window2).alias('rank')).filter(col('rank') <= 3).show()

# There are quite a few common violation codes across the buckets. 
# 38, 36, 21, 14 seem frequent 

+--------------+--------------+--------+----+
|Hour_Frequency|Violation Code|countval|rank|
+--------------+--------------+--------+----+
|            B4|            38|  234449|   1|
|            B4|            36|  193003|   2|
|            B4|            37|  166793|   3|
|            B6|             7|   18382|   1|
|            B6|            40|   17545|   2|
|            B6|            14|   15643|   3|
|            B2|            21|  247422|   1|
|            B2|            14|  109762|   2|
|            B2|            36|   75948|   3|
|            B1|            21|   38283|   1|
|            B1|            40|   28908|   2|
|            B1|            78|   16777|   3|
|            B3|            21|  479606|   1|
|            B3|            36|  388990|   2|
|            B3|            38|  206769|   3|
|            B5|            38|   61211|   1|
|            B5|             7|   45290|   2|
|            B5|            14|   40214|   3|
+--------------+--------------+---

In [46]:
# Get the top 3 occuring violation codes.
parking_nonull.groupby('Violation Code').count().sort(desc("count")).show(5)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+
only showing top 5 rows



In [47]:
# For the three most commonly occurring violation codes, find the most common time of the day 
Top_Violation_Hour = parking_nonull.where((col("Violation Code") == '21') | (col("Violation Code") == '36') | (col("Violation Code") =='38')).groupby('Hour_frequency').count().sort(desc("count")).show(3)

## B3 , B4, B2  are the top hours . this is 5 AM to 4 PM .Most vioaltions during B3 ( 13 to 16 hrs )

+--------------+-------+
|Hour_frequency|  count|
+--------------+-------+
|            B3|1075365|
|            B4| 429891|
|            B2| 350345|
+--------------+-------+
only showing top 3 rows



In [48]:
# Q6. Try and find some seasonality in this data

# First, divide the year into a certain number of seasons, and find the frequencies of tickets for each season
# S1 : Jan - Mar 
# S2 : Apr - Jun  
# S3 : Jul-Sep 
# S4 : Oct - Dec
parking_nonull.show(3)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+----------+----------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|      Date|ViolationTime_Rd|Hour_Frequency|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+----------+----------------+--------------+
|    8478629828| 66623ME|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|                14|             14|        1120AM|2017-06-14|        11:20:00|            B3|
|    5096917368| FZD8593|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|                 0|              0|        0852PM|2017-06-13|        20:52

In [49]:
# Define seasons and caegorise tickets into one of the 4 buckets.
parking_nonull=parking_nonull.selectExpr("*","CASE WHEN month(Date) >= 10 THEN  'S4' WHEN month(Date) >= 7 and month(Date) <= 9  THEN  'S3' WHEN month(Date) >= 4 and month(Date) <= 6  THEN  'S2' ELSE 'S1' END AS Month_Issue")

In [50]:
# Find the three most common violations for each of these seasons.
parking_nonull.createOrReplaceTempView("parkingTable3")
df3=spark.sql('SELECT `Month_Issue`,`Violation Code`, count(`Violation Code`) as countval FROM parkingTable3 group by `Month_Issue`,`Violation Code` order by count(`Violation Code`) desc')
window3 = Window.partitionBy(df3['Month_Issue']).orderBy(df3['countval'].desc())
df3.select('*', rank().over(window3).alias('rank')).filter(col('rank') <= 3).show()

# 21 is commonly occuring violation in all seasons. 

+-----------+--------------+--------+----+
|Month_Issue|Violation Code|countval|rank|
+-----------+--------------+--------+----+
|         S4|            46|     219|   1|
|         S4|            40|     121|   2|
|         S4|            21|     100|   3|
|         S3|            21|     228|   1|
|         S3|            46|     219|   2|
|         S3|            40|     109|   3|
|         S2|            21|  393885|   1|
|         S2|            36|  314525|   2|
|         S2|            38|  255064|   3|
|         S1|            21|  373874|   1|
|         S1|            36|  348240|   2|
|         S1|            38|  287000|   3|
+-----------+--------------+--------+----+



In [51]:
# Q7. Earnings calculation 
# Find the total occurrences of the three most common violation codes.
# 21, 36, 38 - Top violation code..
Violation_Count=parking_nonull.where((col("Violation Code") == '21') | (col("Violation Code") == '36') | (col("Violation Code") =='38')).groupby('Violation Code').count().sort(desc("count"))

In [52]:
Violation_Count.show()

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
+--------------+------+



In [53]:
# Charges for the violation code from the site. 
# violation code 21 - 51$
# violation code 36 - 50$
# violation code 38 - 50$
# Calculate earnings for the top 3 violation codes.
earnings=Violation_Count.selectExpr("*","CASE WHEN (`Violation Code` == 21) THEN  count*51 WHEN (`Violation Code` == 36) THEN count*50 WHEN (`Violation Code` == 38) THEN count*50 ELSE 0 END AS Earnings")

In [54]:
earnings.show()
## Violation code 21 has the highest earnings. 

+--------------+------+--------+
|Violation Code| count|Earnings|
+--------------+------+--------+
|            21|768087|39172437|
|            36|662765|33138250|
|            38|542079|27103950|
+--------------+------+--------+



### Observations:

1. Violation Code 21 is prominent across all seasons and is the top earning for the NYC traffic control.<br>
2. More tickets are issued during the first 6 months of the year than the latter 6 months.<br>
3. More tickets are issued during the business hours/peak hours from 9 AM to 4 PM.<br>
4. NY state has more tickets issued than any other.<br>
5. Precinct 19 is where more tickets are issued by the issuer from the same precinct. 
